<font size="6">Run the LULUCF part of the AFOLU model</font> 

<font size="4">Must be run using the utilities_and_variables.ipynb kernel</font> 

In [119]:
# 
def calculate_and_upload_LULUCF_fluxes(bounds, is_final):

    bounds_str = boundstr(bounds)    # String form of chunk bounds
    tile_id = xy_to_tile_id(bounds[0], bounds[3])    # tile_id in YYN/S_XXXE/W
    chunk_length_pixels = calc_chunk_length_pixels(bounds)   # Chunk length in pixels (as opposed to decimal degrees)

    no_data_val = 255

    
    ### Part 1: Downloads chunks and check for data

    # Dictionary of downloaded layers
    download_dict = {}
    layers = {}

    download_dict = {
        f"{land_cover}_2000": f"{LC_uri}/composite/2000/raw/{tile_id}.tif",
        f"{land_cover}_2005": f"{LC_uri}/composite/2005/raw/{tile_id}.tif",
        f"{land_cover}_2010": f"{LC_uri}/composite/2010/raw/{tile_id}.tif",
        f"{land_cover}_2015": f"{LC_uri}/composite/2015/raw/{tile_id}.tif",
        f"{land_cover}_2020": f"{LC_uri}/composite/2020/raw/{tile_id}.tif",  

        f"{vegetation_height}_2000": f"{LC_uri}/vegetation_height/2000/{tile_id}_vegetation_height_2000.tif",
        f"{vegetation_height}_2005": f"{LC_uri}/vegetation_height/2005/{tile_id}_vegetation_height_2005.tif",
        f"{vegetation_height}_2010": f"{LC_uri}/vegetation_height/2010/{tile_id}_vegetation_height_2010.tif",
        f"{vegetation_height}_2015": f"{LC_uri}/vegetation_height/2015/{tile_id}_vegetation_height_2015.tif",
        f"{vegetation_height}_2020": f"{LC_uri}/vegetation_height/2020/{tile_id}_vegetation_height_2020.tif",  

        agc_2000: f"s3://gfw2-data/climate/carbon_model/carbon_pools/aboveground_carbon/extent_2000/standard/20230222/{tile_id}_Mg_AGC_ha_2000.tif",
        bgc_2000: f"s3://gfw2-data/climate/carbon_model/carbon_pools/belowground_carbon/extent_2000/standard/20230222/{tile_id}_Mg_BGC_ha_2000.tif",
        deadwood_c_2000: f"s3://gfw2-data/climate/carbon_model/carbon_pools/deadwood_carbon/extent_2000/standard/20230222/{tile_id}_Mg_deadwood_C_ha_2000.tif",
        litter_c_2000: f"s3://gfw2-data/climate/carbon_model/carbon_pools/litter_carbon/extent_2000/standard/20230222/{tile_id}_Mg_litter_C_ha_2000.tif",
        soil_c_2000: f"s3://gfw2-data/climate/carbon_model/carbon_pools/soil_carbon/intermediate_full_extent/standard/20231108/{tile_id}_soil_C_full_extent_2000_Mg_C_ha.tif",

        r_s_ratio: f"s3://gfw2-data/climate/carbon_model/BGB_AGB_ratio/processed/20230216/{tile_id}_BGB_AGB_ratio.tif",

        # "drivers": f"s3://gfw2-data/climate/carbon_model/other_emissions_inputs/tree_cover_loss_drivers/processed/drivers_2022/20230407/{tile_id}_tree_cover_loss_driver_processed.tif",
        planted_forest_type_layer: f"s3://gfw2-data/climate/carbon_model/other_emissions_inputs/plantation_type/SDPTv2/20230911/{tile_id}_plantation_type_oilpalm_woodfiber_other.tif", # Originally from gfw-data-lake, so it's in 400x400 windows
        planted_forest_tree_crop_layer: f"s3://gfw2-data/climate/carbon_model/other_emissions_inputs/plantation_simpleType__planted_forest_tree_crop/SDPTv2/20230911/{tile_id}.tif"  # Originally from gfw-data-lake, so it's in 400x400 windows
        # "peat": f"s3://gfw2-data/climate/carbon_model/other_emissions_inputs/peatlands/processed/20230315/{tile_id}_peat_mask_processed.tif",
        # "ecozone": f"s3://gfw2-data/fao_ecozones/v2000/raster/epsg-4326/10/40000/class/gdal-geotiff/{tile_id}.tif",   # Originally from gfw-data-lake, so it's in 400x400 windows 
        # "iso": f"s3://gfw2-data/gadm_administrative_boundaries/v3.6/raster/epsg-4326/10/40000/adm0/gdal-geotiff/{tile_id}.tif",  # Originally from gfw-data-lake, so it's in 400x400 windows
        # "ifl_primary": f"s3://gfw2-data/climate/carbon_model/ifl_primary_merged/processed/20200724/{tile_id}_ifl_2000_primary_2001_merged.tif"
    }

    for year in range(first_year, last_year+1):     # Annual burned area maps start in 2000
        download_dict[f"{burned_area}_{year}"] = f"s3://gfw2-data/climate/carbon_model/other_emissions_inputs/burn_year/burn_year_10x10_clip/ba_{year}_{tile_id}.tif"    

    for year in range(first_year+1, last_year+1):     # Annual forest disturbance maps start in 2001 and ends in 2020
        download_dict[f"{forest_disturbance}_{year}"] = f"{LC_uri}/annual_forest_disturbance/raw/{year}_{tile_id}.tif"     

    # Checks whether tile exists at all. Doesn't try to download chunk if the tile doesn't exist.
    tile_exists = check_for_tile(download_dict, is_final)

    if tile_exists == 0:
        return

    # Note: If running in a local Dask cluster, prints to console may be duplicated. Doesn't happen with a Coiled cluster of the same size (1 worker).
    # Seems to be a problem with local Dask getting overwhelmed by so many futures being created and downloaded from s3. 
    futures = prepare_to_download_chunk(bounds, download_dict, no_data_val)

    # print(futures)

    if not is_final:
        print(f"Waiting for requests for data in chunk {bounds_str} in {tile_id}: {timestr()}")
    
    # Waits for requests to come back with data from S3
    for future in concurrent.futures.as_completed(futures):
        layer = futures[future]
        layers[layer] = future.result()

    # print(layers)
    # print(layers[soil_c_2000].dtype)
    
    # Checks chunk for data. Skips the chunk if it has no data in it.
    data_in_chunk = check_chunk_for_data(layers, f"{land_cover}_", bounds_str, tile_id, no_data_val, is_final)

    if data_in_chunk == 0:
        return

    
    ### Part 2: Create a separate dictionary for each chunk datatype so that they can be passed to Numba as separate arguments.
    ### Numba functions can accept (and return) dictionaries of arrays as long as each dictionary only has arrays of one data type (e.g., uint8, float32)
    ### Note: need to add new code if inputs with other data types are added

    # Initializes empty dictionaries for each type
    uint8_dict_layers = {}
    int16_dict_layers = {}
    float32_dict_layers = {}
    
    # Iterates through the downloaded chunk dictionary and distributes arrays to a separate dictionary for each data type
    for key, array in layers.items():
        if array.dtype == np.uint8:
            uint8_dict_layers[key] = array
        elif array.dtype == np.int16:
            int16_dict_layers[key] = array
        elif array.dtype == np.float32:
            float32_dict_layers[key] = array
        else:
            raise TypeError(f"{key} dtype not in list")

    # print(f"uint8 datasets: {uint8_dict_layers.keys()}")
    # print(f"int16 datasets: {int16_dict_layers.keys()}")
    # print(f"float32 datasets: {float32_dict_layers.keys()}")
    
    # Creates numba-compliant typed dict for each type of array
    typed_dict_uint8 = Dict.empty(
        key_type=types.unicode_type, 
        value_type=types.Array(types.uint8, 2, 'C')  # Assuming 2D arrays of uint8
    )

    typed_dict_int16 = Dict.empty(
        key_type=types.unicode_type, 
        value_type=types.Array(types.int16, 2, 'C')  # Assuming 2D arrays of uint8
    )

    typed_dict_float32 = Dict.empty(
        key_type=types.unicode_type, 
        value_type=types.Array(types.float32, 2, 'C')  # Assuming 2D arrays of float32
    )

    # Populates the numba-compliant typed dicts
    for key, array in uint8_dict_layers.items():
        typed_dict_uint8[key] = array

    for key, array in int16_dict_layers.items():
        typed_dict_int16[key] = array

    for key, array in float32_dict_layers.items():
        typed_dict_float32[key] = array

    
    ### Part 3: Calculates LULUCF fluxes and densities
   
    print(f"Calculating LULUCF fluxes and carbon densities in {bounds_str} in {tile_id}: {timestr()}")

    out_dict_uint16, out_dict_float32 = LULUCF_fluxes(
        typed_dict_uint8, typed_dict_int16, typed_dict_float32 
    )

    # print(out_dict_uint16)
    # print(out_dict_float32)
    # print(f"Average of {list(out_dict_uint16.keys())[0]} is: {list(out_dict_uint16.values())[0].mean()}")

    # Fresh non-Numba-constrained dictionary that stores all numpy arrays.
    # The dictionaries by datatype that are returned from the numba function have limitations on them, 
    # e.g., they can't be combined with other datatypes. This prevents the addition of attributes needed for uploading to s3.
    # So the trick here is to copy the numba-exported arrays into normal Python arrays to which we can do anything in Python.
    
    out_dict_all_dtypes = {}

    # Transfers the dictionaries of numpy arrays for each data type to a new, Pythonic array
    for key, value in out_dict_uint16.items():
        out_dict_all_dtypes[key] = value

    for key, value in out_dict_float32.items():
        out_dict_all_dtypes[key] = value

    # Clear memory of unneeded arrays
    del out_dict_uint16
    del out_dict_float32

    
    ### Part 4: Save numpy as rasters and upload to s3

    # Adds metadata used for uploading outputs to s3 to the dictionary
    for key, value in out_dict_all_dtypes.items():

        data_type = value.dtype.name
        out_pattern = key[:-10]
        year = int(key[-4:])

        # Dictionary with metadata for each array
        out_dict_all_dtypes[key] = [value, data_type, out_pattern, f'{year-5}_{year}']

    save_and_upload_small_raster_set(bounds, chunk_length_pixels, tile_id, bounds_str, out_dict_all_dtypes, is_final)
    
    # Clear memory of unneeded arrays
    del out_dict_all_dtypes

    return f"Success for {bounds_str}: {timestr()}"

In [125]:
# Function to calculate LULUCF fluxes and carbon densities
# Operates pixel by pixel, so uses numba (Python compiled to C++).
@jit(nopython=True)
def LULUCF_fluxes(uint8_layers, int16_layers, float32_layers):

    # Separate dictionaries for output numpy arrays of each datatype, named by output type (e.g., AGC density, BGC removals) and year ([output_type}_[year]).
    # This is because a dictionary in a Numba function cannot have arrays with multiple data types, so each dictionary has to store only one data type,
    # just like inputs to the function.
    out_dict_uint16 = {}
    out_dict_float32 = {}

    end_years = list(range(first_year, last_year+1, interval_years))[1:]
    # end_years = [2005, 2010]

    agc_dens_curr_block = float32_layers[agc_2000]
    bgc_dens_curr_block = float32_layers[bgc_2000]
    deadwood_c_dens_curr_block = float32_layers[deadwood_c_2000]
    litter_c_dens_curr_block = float32_layers[litter_c_2000]
    soil_c_dens_curr_block = int16_layers[soil_c_2000]

    r_s_ratio_block = float32_layers[r_s_ratio]

    for year in end_years:

        # print(year)

        # Creates array for each input
        LC_prev_block = uint8_layers[f"{land_cover}_{year-interval_years}"]
        LC_curr_block = uint8_layers[f"{land_cover}_{year}"]
        veg_h_prev_block = uint8_layers[f"{vegetation_height}_{year-interval_years}"]
        veg_h_curr_block = uint8_layers[f"{vegetation_height}_{year}"]
        planted_forest_type_block = uint8_layers[planted_forest_type_layer]
        planted_forest_tree_crop_block = uint8_layers[planted_forest_tree_crop_layer]

        burned_area_t_4_block = uint8_layers[f"{burned_area}_{year-4}"]
        burned_area_t_3_block = uint8_layers[f"{burned_area}_{year-3}"]
        burned_area_t_2_block = uint8_layers[f"{burned_area}_{year-2}"]
        burned_area_t_1_block = uint8_layers[f"{burned_area}_{year-1}"]
        burned_area_t_block = uint8_layers[f"{burned_area}_{year}"]

        forest_dist_t_4_block = uint8_layers[f"{forest_disturbance}_{year-4}"]
        forest_dist_t_3_block = uint8_layers[f"{forest_disturbance}_{year-3}"]
        forest_dist_t_2_block = uint8_layers[f"{forest_disturbance}_{year-2}"]
        forest_dist_t_1_block = uint8_layers[f"{forest_disturbance}_{year-1}"]
        forest_dist_t_block = uint8_layers[f"{forest_disturbance}_{year}"]

        # Numpy arrays for outputs that don't depend on previous values
        state_out = np.zeros(uint8_layers[f"{land_cover}_2000"].shape).astype('uint16')  
        agc_flux_out_block = np.zeros(float32_layers[agc_2000].shape).astype('float32')
        bgc_flux_out_block = np.zeros(float32_layers[agc_2000].shape).astype('float32')

        
        # Iterates through all pixels in the chunk
        for row in range(LC_curr_block.shape[0]):
            for col in range(LC_curr_block.shape[1]):
                
                LC_prev = LC_prev_block[row, col]
                LC_curr = LC_curr_block[row, col]
                veg_h_prev = veg_h_prev_block[row, col]
                veg_h_curr = veg_h_curr_block[row, col]
                planted_forest_type = planted_forest_type_block[row, col]
                planted_forest_tree_crop = planted_forest_tree_crop_block[row, col]

                # Note: Stacking the burned area rasters using ndstack outside the pixel iteration did not work with numba.
                # So just reading each burned area raster separately.
                burned_area_t_4 = burned_area_t_4_block[row, col]
                burned_area_t_3 = burned_area_t_3_block[row, col]
                burned_area_t_2 = burned_area_t_2_block[row, col]
                burned_area_t_1 = burned_area_t_1_block[row, col]
                burned_area_t = burned_area_t_block[row, col]
                burned_area_last = max([burned_area_t_4, burned_area_t_3, burned_area_t_2, burned_area_t_1, burned_area_t])  # Most recent year with burned area during the interval

                forest_dist_t_4 = forest_dist_t_4_block[row, col]
                forest_dist_t_3 = forest_dist_t_3_block[row, col]
                forest_dist_t_2 = forest_dist_t_2_block[row, col]
                forest_dist_t_1 = forest_dist_t_1_block[row, col]
                forest_dist_t = forest_dist_t_block[row, col]
                forest_dist_last = max([forest_dist_t_4, forest_dist_t_3, forest_dist_t_2, forest_dist_t_1, forest_dist_t])  # Most recent year with forest disturbance during the interval   

                agc_dens_curr = agc_dens_curr_block[row, col]
                bgc_dens_curr = bgc_dens_curr_block[row, col]
                deadwood_c_dens_curr = deadwood_c_dens_curr_block[row, col]
                litter_c_dens_curr = litter_c_dens_curr_block[row, col]
                soil_c_dens_curr = soil_c_dens_curr_block[row, col]

                r_s_ratio_cell = r_s_ratio_block[row, col]

                tree_prev = (veg_h_prev >= tree_threshold)
                tree_curr = (veg_h_curr >=  tree_threshold)
                tall_veg_prev = (((LC_prev >= tree_dry_min_height_code) and (LC_prev <= tree_dry_max_height_code)) or
                                        ((LC_prev >= tree_wet_min_height_code) and (LC_prev <= tree_wet_max_height_code)))
                tall_veg_curr = (((LC_curr >= tree_dry_min_height_code) and (LC_curr <= tree_dry_max_height_code)) or
                                       ((LC_curr >= tree_wet_min_height_code) and (LC_curr <= tree_wet_max_height_code)))
                short_med_veg_prev = (((LC_prev >= 2) and (LC_prev <= 26)) or
                                       ((LC_prev >= 102) and (LC_prev <= 126)))
                short_med_veg_curr = (((LC_curr >= 2) and (LC_curr <= 26)) or
                                       ((LC_curr >= 102) and (LC_curr <= 126)))

                sig_height_loss_prev_curr = (veg_h_prev-veg_h_curr >= sig_height_loss_threshold) 

                
                ### Tree gain
                if (not tree_prev) and (tree_curr):                  # Non-tree converted to tree (1)    ##TODO: Include mangrove exception.
                    if planted_forest_type == 0:                     # New non-SDPT trees (11)
                        if not tall_veg_curr:                        # New trees outside forests (111)
                            state_out[row, col] = 111
                            agc_rf = 2.8
                            agc_flux_out_block[row, col] = (agc_rf*interval_years)*-1
                            agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]   
                            bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                            bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col]   
                        else:                                        # New terrestrial natural forest (112)
                            state_out[row, col] = 112
                            agc_rf = 5.6
                            agc_flux_out_block[row, col] = (agc_rf*interval_years)*-1
                            agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                            bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                            bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                    else:                                            # New SDPT trees (12)
                        state_out[row, col] = 12 
                        agc_rf = 10
                        agc_flux_out_block[row, col] = (agc_rf*interval_years)*-1
                        agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]   
                        bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                        bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                
                ### Tree loss
                elif (tree_prev) and (not tree_curr):                # Tree converted to non-tree (2)    ##TODO: Include forest disturbance condition.  ##TODO: Include mangrove exception.
                    if planted_forest_type == 0:                     # Full loss of non-SDPT trees (21)
                        if not tall_veg_prev:                        # Full loss of trees outside forests (211)
                            if burned_area_last == 0:                # Full loss of trees outside forests without fire (2111)
                                state_out[row, col] = 2111
                                agc_ef = 0.8
                                agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                            else:                                    # Full loss of trees outside forests with fire (2112)
                                state_out[row, col] = 2112
                                agc_ef = 0.6
                                agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                        else:                                        # Full loss of natural forest (212)
                            if LC_curr == cropland:                  # Full loss of natural forest converted to cropland (2121)
                                if burned_area_last == 0:            # Full loss of natural forest converted to cropland, not burned (21211)
                                    state_out[row, col] = 21211
                                    agc_ef = 0.3
                                    agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                    agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                    bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                    bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                                else:                                # Full loss of natural forest converted to cropland, burned (21212)
                                    state_out[row, col] = 21212
                                    agc_ef = 0.3
                                    agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                    agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                    bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                    bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                            elif short_med_veg_curr:                 # Full loss of natural forest converted to short or medium vegetation (2122)
                                if burned_area_last == 0:            # Full loss of natural forest converted to short or medium vegetation, not burned (21221)
                                    state_out[row, col] = 21221
                                    agc_ef = 0.9
                                    agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                    agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                    bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                    bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                                else:                                # Full loss of natural forest converted to short or medium vegetation, burned (21222)
                                    state_out[row, col] = 21222
                                    agc_ef = 0.3
                                    agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                    agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                    bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                    bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                            elif LC_curr == builtup:                 # Full loss of natural forest converted to builtup (2123)
                                if burned_area_last == 0:            # Full loss of natural forest converted to builtup, not burned (21231)
                                    state_out[row, col] = 21231
                                    agc_ef = 0.3
                                    agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                    agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                    bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                    bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                                else:                                # Full loss of natural forest converted to builtup, burned (21232)
                                    state_out[row, col] = 21232
                                    agc_ef = 0.3
                                    agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                    agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                    bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                    bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                            else:                                    # Full loss of natural forest converted to anything else (2124)
                                if burned_area_last == 0:            # Full loss of natural forest converted to builtup, not burned (21241)
                                    state_out[row, col] = 21241
                                    agc_ef = 0.3
                                    agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                    agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                    bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                    bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                                else:                                # Full loss of natural forest converted to builtup, burned (21242)
                                    state_out[row, col] = 21242       
                                    agc_ef = 0.3
                                    agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                    agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                    bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                    bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                    else:                                            # Full loss of SDPT trees (22)
                        if LC_curr == cropland:                      # Full loss of SDPT converted to cropland (221)
                            if burned_area_last == 0:                # Full loss of SDPT converted to cropland, not burned (2211)
                                state_out[row, col] = 2211
                                agc_ef = 0.3
                                agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                            else:                                    # Full loss of SPDPT converted to cropland, burned (2212)
                                state_out[row, col] = 2212
                                agc_ef = 0.3
                                agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                        elif short_med_veg_curr:                     # Full loss of SDPT converted to short or medium vegetation (222)
                            if burned_area_last == 0:                # Full loss of SDPT converted to short or medium vegetation, not burned (2221)
                                state_out[row, col] = 2221
                                agc_ef = 0.3
                                agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                            else:                                    # Full loss of SDPT converted to short or medium vegetation, burned (2222)
                                state_out[row, col] = 2222
                                agc_ef = 0.3
                                agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                        elif LC_curr == builtup:                     # Full loss of SDPT converted to builtup (223)
                            if planted_forest_tree_crop == 1:        # Full loss of SDPT planted forest to builtup (2231)
                                if burned_area_last == 0:            # Full loss of SDPT planted forest converted to builtup, not burned (22311)
                                    state_out[row, col] = 22311
                                    agc_ef = 0.3
                                    agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                    agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                    bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                    bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                                else:                                # Full loss of SDPT planted forest converted to builtup, burned (22312)
                                    state_out[row, col] = 22312
                                    agc_ef = 0.3
                                    agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                    agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                    bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                    bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                            else:                                    # Full loss of SDPT tree crop to builtup (2232)
                                if burned_area_last == 0:            # Full loss of SDPT tree crop converted to builtup, not burned (22321)
                                    state_out[row, col] = 22321
                                    agc_ef = 0.3
                                    agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                    agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                    bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                    bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                                else:                                # Full loss of SDPT tree crop converted to builtup, burned (22322)
                                    state_out[row, col] = 22322         
                                    agc_ef = 0.3
                                    agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                    agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                    bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                    bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                        else:                                        # Full loss of SDPT converted to anything else (224)
                            if burned_area_last == 0:                # Full loss of SDPT converted to builtup, not burned (2241)
                                state_out[row, col] = 2241
                                agc_ef = 0.3
                                agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                            else:                                    # Full loss of SDPT converted to builtup, burned (2242)
                                state_out[row, col] = 2242
                                agc_ef = 0.3
                                agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                
                ### Trees remaining trees
                elif (tree_prev) and (tree_curr):                    # Trees remaining trees (3)    ##TODO: Include mangrove exception.
                    if forest_dist_last == 0:                        # Trees without stand-replacing disturbances in the last interval (31)
                        if planted_forest_type == 0:                 # Non-planted trees without stand-replacing disturbance in the last interval (311)
                            if not tall_veg_curr:                    # Trees outside forests without stand-replacing disturbance in the last interval (3111)
                                if not sig_height_loss_prev_curr:    # Stable trees outside forests (31111)
                                    state_out[row, col] = 31111
                                else:                                # Partially disturbed trees outside forests (31112)
                                    if burned_area_last == 0:        # Partially disturbed trees outside forests without fire (311121)
                                        state_out[row, col] = 311121
                                    else:
                                        state_out[row, col] = 311122 # Partially disturbed trees outside forests with fire (311122)
                            
                    else:
                        state_out[row, col] = 32
                    
                else:                                                # Not covered in above branches
                    state_out[row, col] = 65534
        
        # Adds the output arrays to the dictionary with the appropriate data type
        # Outputs need .copy() so that previous intervals' arrays in dicationary aren't overwritten because arrays in dictionaries are mutable (courtesy of ChatGPT).
        year_range = f"{year-interval_years}_{year}"
        out_dict_uint16[f"{land_state_pattern}_{year_range}"] = state_out.copy()  
        out_dict_float32[f"{agc_dens_pattern}_{year_range}"] = agc_dens_curr_block.copy()
        out_dict_float32[f"{bgc_dens_pattern}_{year_range}"] = bgc_dens_curr_block.copy()
        out_dict_float32[f"{agc_flux_pattern}_{year_range}"] = agc_flux_out_block.copy()
        out_dict_float32[f"{bgc_flux_pattern}_{year_range}"] = bgc_flux_out_block.copy()

    return out_dict_uint16, out_dict_float32

In [122]:
%%time

## Create LULUCF flux and carbon stock 2x2 deg rasters 

## Area to analyze
## chunk_params arguments: W, S, E, N, chunk size (degrees)
# chunk_params = [-180, -60, 180, 80, 2]  # entire world
# chunk_params = [-10, 40, 20, 70, 1]    # 30x30 deg (70N_010W), 900 chunks

# chunk_params = [-10, 60, 0, 70, 1]    # 10x10 deg (70N_010W), 100 chunks
# chunk_params = [-10, 65, -5, 70, 1]    # 5x5 deg (70N_010W), 25 chunks
# chunk_params = [-10, 68, -8, 70, 1]    # 2x2 deg (70N_010W), 4 chunks
# chunk_params = [-10, 69, -9, 70, 1]    # 1x1 deg (70N_010W), 1 chunk

# chunk_params = [10, 40, 20, 50, 2]    # 10x10 deg (50N_010E), 25 chunks
# chunk_params = [10, 40, 20, 50, 10]    # 10x10 deg (50N_010E), 1 chunk
# chunk_params = [10, 46, 14, 50, 2]   # 4x4 deg, 4 chunks
# chunk_params = [110, -10, 114, -6, 2]   # 4x4 deg, 4 chunks
# chunk_params = [10, 48, 12, 50, 1]   # 2x2 deg, 4 chunks
# chunk_params = [10, 49, 11, 50, 1]   # 1x1 deg, 1 chunk
# chunk_params = [10, 49, 11, 50, 0.5] # 1x1 deg, 4 chunks
# chunk_params = [10, 49.5, 10.5, 50, 0.25] # 0.5x0.5 deg, 4 chunks
# chunk_params = [10, 42, 11, 43, 0.5] # 1x1 deg, 4 chunks (some GLCLU code=254 for ocean and some land, so data should be output)
# chunk_params = [10, 49.75, 10.25, 50, 0.25] # 0.25x0.25 deg, 1 chunk (has data, no fire)
chunk_params = [15, 41.75, 15.25, 42, 0.25] # 0.25x0.25 deg, 1 chunk (has data with fire)

# # Range of no-data cases for testing
# chunk_params = [20, 69.75, 20.25, 70, 0.25] # 0.25x0.25 deg, 1 chunk (tile exists for GLCLU but not all other inputs, e.g., fire)
# chunk_params = [110, -10, 120, 0, 2]    # 10x10 deg (00N_110E), 25 chunks (all chunks have land and should be output)
# chunk_params = [110, -20, 120, -10, 2]    # 10x10 deg (00N_110E), 25 chunks (all chunks have land and should be output)
# chunk_params = [0, 79.75, 0.25, 80, 0.25] # 0.25x0.25 deg, 1 chunk (no 80N_000E tile-- no data)
# chunk_params = [112, -12, 116, -8, 2]   # 2x2 deg, 1 chunk (bottom of Java, has data but mostly ocean)
# chunk_params = [10.875, 41.75, 11, 42, 0.25] # 0.25x0.25 deg, 1 chunk (entirely GLCLU code=255 for ocean, so no actual data-- nothing should be be output)
# chunk_params = [-10, 21.75, -9.75, 22, 0.25] # 0.25x0.25 deg, 1 chunk (has data but entirely desert (fully GLCLU code=0))
# chunk_params = [10, 49.75, 10.25, 50, 0.25] # 0.25x0.25 deg, 1 chunk (has data)


# Makes list of chunks to analyze
chunks = get_chunk_bounds(chunk_params)  
print("Processing", len(chunks), "chunks")
# print(chunks)

# Determines if the output file names for final versions of outputs should be used
is_final = False
if len(chunks) > 90:
    is_final = True
    print("Running as final model.")

# Creates list of tasks to run (1 task = 1 chunk for all years)
delayed_result = [dask.delayed(calculate_and_upload_LULUCF_fluxes)(chunk, is_final) for chunk in chunks]

# Actually runs analysis
results = dask.compute(*delayed_result)
results

Processing 1 chunks
Tile id 50N_010E exists. Proceeding.
Requesting data in chunk 15_42_15_42 in 50N_010E: 20240417_20_58_12
Waiting for requests for data in chunk 15_42_15_42 in 50N_010E: 20240417_20_58_28
Data in chunk 15_42_15_42. Proceeding.
Calculating LULUCF fluxes and carbon densities in 15_42_15_42 in 50N_010E: 20240417_20_58_28
Saving 15_42_15_42 in 50N_010E for 2000_2005: 20240417_20_58_32
Uploading 15_42_15_42 in 50N_010E for 2000_2005 to climate/AFOLU_flux_model/LULUCF/outputs/land_state_node/2000_2005/1000_pixels/20240417: 20240417_20_58_32
Saving 15_42_15_42 in 50N_010E for 2005_2010: 20240417_20_58_32
Uploading 15_42_15_42 in 50N_010E for 2005_2010 to climate/AFOLU_flux_model/LULUCF/outputs/land_state_node/2005_2010/1000_pixels/20240417: 20240417_20_58_32
Saving 15_42_15_42 in 50N_010E for 2010_2015: 20240417_20_58_32
Uploading 15_42_15_42 in 50N_010E for 2010_2015 to climate/AFOLU_flux_model/LULUCF/outputs/land_state_node/2010_2015/1000_pixels/20240417: 20240417_20_58_3

('Success for 15_42_15_42: 20240417_20_58_34',)

In [ ]:
# Execute without Dask
calculate_and_upload_LULUCF_fluxes([10, 49.75, 10.25, 50], is_final=False)

In [ ]:
def get_tile_dataset_rio(uri, bounds, chunk_length_pixels, no_data_val):

    bounds_str = boundstr(bounds)

    try:
        print("trying")
        with rasterio.open(uri) as ds:
            window = rasterio.windows.from_bounds(*bounds, ds.transform)
            data = ds.read(1, window=window)
    except:
        print("excepting")
        data = np.full((chunk_length_pixels, chunk_length_pixels), no_data_val)

    return data

get_tile_dataset_rio("s3://gfw2-data/climate/carbon_model/other_emissions_inputs/burn_year/burn_year_10x10_clip/ba_2017_70N_020E.tif", [20, 69.75, 20.25, 70], 1000, 255)